## Loading libraries

In [1]:
from aiida import load_profile, orm
## Indicate your profile name here
your_profile_name = 'diffusion'
load_profile(your_profile_name) 
from aiida.plugins import WorkflowFactory
from aiida.engine import submit

## Loading the code and structure

In [22]:
## AiiDA code that will run Quantum ESPRESSO on the cluster
code = orm.load_code(label='pw_7_eiger@eiger')

## default parameters
time, num_machines, num_mpiprocs_per_machine, num_cores_per_mpiproc, npool = 43200, 2, 128, 1, 8

## overrides defining pseudopotentials 
overrides = {"ocv_relax":{
                "base":{
                    "pseudo_family": "SSSP/1.1.2/PBEsol/efficiency",
                    "pw":{
                        "parallelization":{
                            "npool": npool},}},
                "base_final_scf":{
                    "pseudo_family": "SSSP/1.1.2/PBEsol/efficiency",
                    "pw":{
                        "parallelization":{
                            "npool": npool},}}},
            "scf":{
                "pseudo_family": "SSSP/1.1.2/PBEsol/efficiency",
                "pw":{
                    "parallelization":{
                        "npool": int(npool/num_machines)},}},}

## Load one of the 3 bundled test structures here
structure = orm.load_node('3dd4a60a-a5d5-48b6-a8b1-3e082664622a') # LiFePO4 
# structure = orm.load_node('096d9d96-7f66-442b-a27f-2660572808ea') # LiCoO2
# structure = orm.load_node('c0537852-7300-43ea-af99-7813eea8a167') # MgMo3S4 

## Load either the Li or Mg bulk cation structure here depending on the above ``structure``
bulk_cation_structure = orm.load_node('faea3c48-789f-4076-af73-0cf9242bb7b2 ') # Li
# bulk_cation_structure = orm.load_node('df0a8dda-3d84-44d6-8b97-069ee8d0a62e') # Mg

<WorkChainNode: uuid: ce46e557-c646-4305-94ed-45e52995bdc8 (pk: 110535) (aiida.workflows:quantumespresso.ocv.ocvwc)>

## Launching OCVWorkChain

### Using a default protocol

In [21]:
## Defining the OCVWorkChain with some suggested parameters

def submit_OCV_workchain(structure, code, bulk_cation_structure=None, discharged_unitcell_relaxed=None, charged_unitcell_relaxed=None, time=time, num_machines=num_machines, 
num_mpiprocs_per_machine=num_mpiprocs_per_machine, num_cores_per_mpiproc=num_cores_per_mpiproc):

    OCVWorkChain = WorkflowFactory('quantumespresso.ocv.ocvwc')

    builder = OCVWorkChain.get_builder_from_protocol(code=code, structure=structure, bulk_cation_structure=bulk_cation_structure, 
    discharged_unitcell_relaxed=discharged_unitcell_relaxed, charged_unitcell_relaxed=charged_unitcell_relaxed, overrides=overrides, protocol='fast')    

    builder.update({'clean_workdir':orm.Bool(False)})

    builder.ocv_relax.base.pw.parallelization = orm.Dict(dict={'npool': npool})
    builder.ocv_relax.base_final_scf.pw.parallelization = orm.Dict(dict={'npool': npool})

    builder.ocv_relax['base']['pw']['metadata']['options'].update({'max_wallclock_seconds': time})
    builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_machines'] = num_machines
    builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_mpiprocs_per_machine'] = num_mpiprocs_per_machine
    builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_cores_per_mpiproc'] = num_cores_per_mpiproc
    builder.ocv_relax['base']['pw']['parameters']['ELECTRONS']['electron_maxstep'] = 200

    builder.ocv_relax['base_final_scf']['pw']['metadata']['options'].update({'max_wallclock_seconds': time})
    builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_machines'] = num_machines
    builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_mpiprocs_per_machine'] = num_mpiprocs_per_machine
    builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_cores_per_mpiproc'] = num_cores_per_mpiproc
    builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['electron_maxstep'] = 250

    # Keep a small value as it will probably finish in a few minutes even on a small cluster
    if bulk_cation_structure:
        builder.scf['pw']['metadata']['options']['max_wallclock_seconds'] = 1800
        builder.scf['pw']['metadata']['options']['resources']['num_machines'] = 1
        builder.scf['pw']['metadata']['options']['resources']['num_mpiprocs_per_machine'] = num_mpiprocs_per_machine
        builder.scf['pw']['metadata']['options']['resources']['num_cores_per_mpiproc'] = num_cores_per_mpiproc
        builder.scf['pw']['parameters']['ELECTRONS']['electron_maxstep'] = 100
        builder.scf.pw.parallelization = orm.Dict(dict={'npool': int(npool/num_machines)})

    builder.ocv_parameters['distance'] = 8.0

    # Don't forget to change following to 'Mg' incase you use MgMo3S4 as the structure
    # otherwise leave 'Li' or empty as 'Li' is the defult value
    builder.ocv_parameters['cation'] = 'Li'

    return builder

In [ ]:
## Submitting the OCVWorkChain

builder = submit_OCV_workchain(structure=structure, code=code, bulk_cation_structure=bulk_cation_structure)
# Submitting the builder to launch the WorkChain
submit(builder)

### Using a json file as default input 

In [ ]:
## Using a json file which contains all the necessary inputs to launch the OCVWorkChain

OCVWorkChain = WorkflowFactory('quantumespresso.ocv.ocvwc')

builder = OCVWorkChain.get_builder_from_json('json_inputs/Li_olivine.json', overrides=overrides)

builder.ocv_relax['base']['pw']['parameters']['ELECTRONS']['electron_maxstep'] = 200

builder.ocv_relax['base']['pw']['metadata']['options']['max_wallclock_seconds'] = time
builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_machines'] = num_machines
builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_mpiprocs_per_machine'] = num_mpiprocs_per_machine
builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_cores_per_mpiproc'] = num_cores_per_mpiproc

builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['max_wallclock_seconds'] = time
builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_machines'] = num_machines
builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_mpiprocs_per_machine'] = num_mpiprocs_per_machine
builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_cores_per_mpiproc'] = num_cores_per_mpiproc
builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['electron_maxstep'] = 250

builder.ocv_relax['max_meta_convergence_iterations'] = orm.Int(5)

# Keep a small value as it will probably finish in a few minutes even on a small cluster
builder.scf['pw']['metadata']['options']['max_wallclock_seconds'] = 1800
builder.scf['pw']['metadata']['options']['resources']['num_machines'] = 1
builder.scf['pw']['metadata']['options']['resources']['num_mpiprocs_per_machine'] = num_mpiprocs_per_machine
builder.scf['pw']['metadata']['options']['resources']['num_cores_per_mpiproc'] = num_cores_per_mpiproc
builder.scf['pw']['parameters']['ELECTRONS']['electron_maxstep'] = 100

builder.ocv_parameters['distance'] = 8.0

# Submitting the builder to launch the WorkChain
submit(builder)